In [10]:
import pandas as pd
import pyodbc
import pgeocode
import useful_functions as use

In [3]:
username = 'vigrose'
password_edw = 'Ravenclaw~10946'
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)
AMAEDW = pyodbc.connect(w)
password_aims = 'DQA8K@T7pXEqEt'
s = "DSN=aims_prod; UID={}; PWD={}".format(username, password_aims)
informix = pyodbc.connect(s)

In [4]:
query_3 = \
    f'''
    SELECT
    U.ENTITY_ID,
    U.COMM_ID,
    U.SRC_CAT_CODE,
    p.addr_line2, 
    p.addr_line1, 
    p.city_cd, 
    p.state_cd, 
    p.zip
    FROM
    informix.ENTITY_COMM_AT U join informix.POST_ADDR_AT P on U.COMM_ID = P.COMM_ID
    WHERE
    U.SRC_CAT_CODE = 'NAT-DOWN'
    ;
    '''
nat_down_addresses = pd.read_sql(con=informix, sql=query_3)
nat_down_addresses.head()

,entity_id,comm_id,src_cat_code,addr_line2,addr_line1,city_cd,state_cd,zip
0,21918360,26491704,NAT-DOWN,951 NW 13th St,Ste 4E,Boca Raton,FL,33486
1,1234619,27931222,NAT-DOWN,1111 E McDowell Rd,,Phoenix,AZ,85006
2,1234619,27931221,NAT-DOWN,14502 W Meeker Blvd,,Sun City,AZ,85375
3,1234645,27553719,NAT-DOWN,300 N Airport Rd,Ste 2,Jasper,AL,35504
4,1234658,27544478,NAT-DOWN,208 McFarland Cir N,Ste 100,Tuscaloosa,AL,35406


In [6]:
query_3 = \
    f'''
    SELECT
    U.ENTITY_ID,
    U.COMM_ID,
    U.SRC_CAT_CODE,
    p.addr_line2, 
    p.addr_line1, 
    p.city_cd, 
    p.state_cd, 
    p.zip
    FROM
    informix.ENTITY_COMM_AT U join informix.POST_ADDR_AT P on U.COMM_ID = P.COMM_ID
    WHERE
    U.SRC_CAT_CODE = 'CRED-ADDR'
    ;
    '''
cred_addresses = pd.read_sql(con=informix, sql=query_3)
cred_addresses.head()

,entity_id,comm_id,src_cat_code,addr_line2,addr_line1,city_cd,state_cd,zip
0,1234658,26789742,CRED-ADDR,1653 Temple Ave N,P O Drawer 710,Fayette,AL,35555
1,1234671,26577992,CRED-ADDR,3719 Dauphin St Frnt,,Mobile,AL,36608
2,1234671,26807023,CRED-ADDR,2451 University Hospital Dr,Mastin Bldg - 6th Floor,Mobile,AL,36617
3,1234892,26772747,CRED-ADDR,1613 N McKenzie St,,Foley,AL,36535
4,1234892,26789730,CRED-ADDR,1653 Temple Ave N,P O Drawer 710,Fayette,AL,35555


In [8]:
informix.close()

In [21]:
ENTITY_TO_ME['entity_id'] = ENTITY_TO_ME.ENTITY_ID.astype(int)

In [12]:
QUERY = \
        """
        SELECT DISTINCT 
        M.ME, E.ENTITY_ID 
        FROM 
        AMAEDW.PARTY_ID_TO_ME_VW M, AMAEDW.PARTY_ENTITY_VW E 
        WHERE 
        M.PARTY_ID=E.PARTY_ID
        """
    #Execute queries
w = "DSN=PRDDW; UID={}; PWD={}".format(username, password_edw)

ENTITY_TO_ME = pd.read_sql(con=AMAEDW, sql=QUERY)
ENTITY_TO_ME['entity_id'] = ENTITY_TO_ME.ENTITY_ID.astype(int)
load = pd.merge(nat_down_addresses, ENTITY_TO_ME, on='entity_id')

In [26]:
load = pd.merge(nat_down_addresses, ENTITY_TO_ME, on='entity_id')

In [25]:
phones = pd.read_csv('../../Data/DataGov/Phone_Sample.csv')

In [34]:
phones['zip'] = [use.fix_zipcode(x) for x in phones.addr_zip]
phones['ME'] = use.fix_me(phones.ME)

In [36]:
phones[['ME','addr_line_2', 'addr_line_1','addr_city', 'addr_state', 'zip']]

,ME,addr_line_2,addr_line_1,addr_city,addr_state,zip
0,49575000010,SUITE 4E,951 NW 13TH ST,BOCA RATON,FL,33486
1,49575000010,SUITE 120,840 US HWY 1,NORTH PALM BEACH,FL,33408
2,02106050565,SUITE 4000,7777 HENNESSY BLVD,BATON ROUGE,LA,70808
3,02106050565,SUITE 3000,7777 HENNESSY BLVD,BATON ROUGE,LA,70808
4,04113040195,10A,980 BEAVER GRADE RD,CORAOPOLIS,PA,15108
...,...,...,...,...,...,...
447247,05104181637,SANTA CLAVA VLY MED,751 S BASCOM AVE,SAN JOSE,CA,95128
447248,01104140765,E 101,6798 CROSSWINDS DR,SAINT PETERSBURG,FL,33710
447249,66501150721,SUITE 230,2409 N PATTERSON ST,VALDOSTA,GA,31602
447250,06509140021,FL 4,4 WESTCHESTER PARK DR,WHITE PLAINS,NY,10604


In [40]:
load[['ME','addr_line_2', 'addr_line_1','addr_city', 'addr_state', 'zip']]

,ME,addr_line_2,addr_line_1,addr_city,addr_state,zip
0,49575000010,STE 4E,951 NW 13TH ST,BOCA RATON,FL,33486
1,49575000010,STE 120,840 US HIGHWAY 1,NORTH PALM BEACH,FL,33408
2,49575000010,,2330 S CONGRESS AVE,PALM SPRINGS,FL,33406
3,00102000048,,1111 E MCDOWELL RD,PHOENIX,AZ,85006
4,00102000048,,14502 W MEEKER BLVD,SUN CITY,AZ,85375
...,...,...,...,...,...,...
1330849,02775150672,STE 2220,1105 CENTRAL EXPY N,ALLEN,TX,75013
1330850,02775150672,STE 500,4521 MEDICAL CTR DR,MCKINNEY,TX,75069
1330851,02775150672,STE 205,5425 W SPRING CREEK PKWY,PLANO,TX,75024
1330852,02775150672,STE 500,4521 MEDICAL CTR DR,MCKINNEY,TX,75069


In [48]:
load['addr_line_1'] = [x.upper().strip() for x in load.addr_line2]
load['addr_line_2'] = [x.upper().strip() for x in load.addr_line1]
load['addr_city'] = [x.upper().strip() for x in load.city_cd]
load['zip'] = [x.strip() for x in load.zip]
load['addr_state'] = load.state_cd

AttributeError: 'NoneType' object has no attribute 'strip'

In [69]:
all_this = pd.merge(load, phones, on=['ME','addr_line_1','addr_city', 'addr_state']).drop_duplicates()

In [68]:
[['ME','entity_id','comm_id','src_cat_code','phone_number']].to_csv('../../PhoneAppend/National_Downloadable_File_Load_2021-11-15.csv', index=False)

,ME,entity_id,comm_id,src_cat_code,phone_number
0,49575000010,21918360,26491704,NAT-DOWN,5619239906
1,00102000064,1234645,27553719,NAT-DOWN,2052217099
2,00102000072,1234658,27544478,NAT-DOWN,2053457000
3,00102000196,1234814,27650757,NAT-DOWN,2514458242
4,00102000196,1234814,27650758,NAT-DOWN,2514458242
...,...,...,...,...,...
382811,04177121881,65910442,28156187,NAT-DOWN,2153456050
382812,04177112173,66081132,28156398,NAT-DOWN,5705016730
382813,04178161682,67154204,28156537,NAT-DOWN,4356281641
382814,02775150672,67582177,28156576,NAT-DOWN,9725628383


In [86]:
all_this.to_csv('../../Data/DataGov/Load_With_Phones', index=False)

In [ ]:
412-2624-91
225-766-744
561-9239-91

In [66]:
phones['phone_number'] = [use.fix_phone(x) for x in phones.phone]

In [ ]:
def get_present_employment():
    present_employment_key = {
        11: 'Self-Employed Solo Practice',
        13: 'Two Physician Practice-Full Or Part Owner',
        21: 'Other-Patient Care',
        22: 'Locum Tenens',
        30: 'Group Practice',
        35: 'HMO',
        40: 'Medical School',
        50: 'Non-Government Hospital',
        63: 'City/County/State Government-Hospital',
        64: 'City/County/State Government-Other Than Hospital',
        81: 'Federal Government-Hospital/Army',
        82: 'Federal Government-Hospital/Navy',
        83: 'Federal Government-Hospital/Air Force',
        84: 'Federal Government-Hospital/Usphs',
        85: 'Federal Government-Hospital/Vet Admin',
        86: 'Federal Government-Hospital/Other Agency',
        101: 'Other/Non-Patient Care',
        110: 'No Classification'
    }
    return present_employment_key

def humach_samplify(data):
    present_employment_key = get_present_employment()
    data['DESCRIPTION'] = [present_employment_key[x] for x in data.PE_CD]
    data = data[data.TELEPHONE_NUMBER!='None']
    new_columns = {
        'ADDR_1_POLO':'POLO_MAILING_LINE_2',
        'ADDR_2_POLO':'POLO_MAILING_LINE_1',
        'CITY_POLO':'POLO_CITY',
        'STATE_POLO':'POLO_STATE',
        'ZIP_POLO':'POLO_ZIP'}
    humach_columns = [
        'ME',
        'FIRST_NAME',
        'MIDDLE_NAME',
        'LAST_NAME',
        'SUFFIX',
        'ADDR_2_POLO',
        'ADDR_1_POLO',
        'CITY_POLO',
        'STATE_POLO',
        'ZIP_POLO',
        'TELEPHONE_NUMBER',
        'PRIM_SPEC_CD',
        'DESCRIPTION',
        'PE_CD',
        'FAX_NUMBER'
    ]
    humach_data = data[humach_columns].rename(columns = new_columns)
    return humach_data

In [ ]:
def clean_file(all_that, today):
    '''Clean and format data'''
    LOGGER.info('Cleaning file')
    all_that = all_that.drop_duplicates(['ME'])
    all_that = remove_whitespaces(all_that)
    all_that['addr_plus4'] = ''
    all_that['addr_zip'] = fix_zipcodes(all_that.ZIP_POLO)
    all_that['addr_country'] = ''
    all_that['ADDR_2_POLO'] = ['' if x=='None' else x for x in all_that.ADDR_2_POLO]
    all_that['ADDR_3_POLO'] = ['' if x=='None' else x for x in all_that.ADDR_3_POLO]
    all_that['usage'] = 'PP'
    all_that['load_type_ind'] = 'R'
    all_that['addr_type'] = 'OF'
    all_that['source'] = 'POLO-PPD'
    all_that['source_dtm'] = today
    col = get_col()
    all_that = all_that[['ENTITY',
                         'ME',
                         'COMM_ID',
                         'usage',
                         'load_type_ind',
                         'addr_type',
                         'ADDR_1_POLO',
                         'ADDR_2_POLO',
                         'ADDR_3_POLO',
                         'CITY_POLO',
                         'STATE_POLO',
                         'addr_zip',
                         'addr_plus4',
                         'addr_country',
                         'source',
                         'source_dtm',
                         ]].rename(columns=col)
    return all_that

In [78]:
all_this.columns

Index(['entity_id', 'comm_id', 'src_cat_code', 'addr_line2', 'addr_line1',
       'city_cd', 'state_cd', 'zip_x', 'ME', 'ENTITY_ID', 'addr_line_1',
       'addr_line_2_x', 'addr_city', 'addr_state', 'me#', 'usage', 'load_type',
       'addr_type', 'addr_line_2_y', 'addr_line_3', 'addr_zip', 'addr_plus4',
       'addr_country', 'source', 'source_dtm', 'adrs_id', 'phone', 'zip_y',
       'phone_number'],
      dtype='object')

In [81]:
for row in all_this.itertuples():
    if row.zip_y not in row.zip_x:
        print(f'{row.addr_line_1} {row.addr_city}')
        print(f'{row.addr_line_2_y} is not {row.addr_line_2_x}')
        print(f'{row.zip_y} is not {row.zip_x}')

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
102 MICAH WAY SCOTTSBORO
SUITE 1107 is not STE 1107
35768 is not 35769     
4300 W MAIN ST DOTHAN
SUITE 102 is not STE 102
36301 is not 36305     
1088 9TH AVE SW BESSEMER
SUITE 104 is not STE 104
35023 is not 35022     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
4300 W MAIN ST DOTHAN
SUITE 102 is not STE 102
36301 is not 36305     
1717 N E ST PENSACOLA
SUITE 308 is not STE 308
32505 is not 32501     
1717 N E ST PENSACOLA
SUITE 308 is not STE 308
32505 is not 32501     
1088 9TH AVE SW BESSEMER
SUITE 104 is not STE 104
35023 is not 35022     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
4300 W MAIN ST DOTHAN
SUITE 102 is not STE

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGE

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
1600 DIVISADERO ST SAN FRANCISCO
MCB302 is not MCB302
94115 is not 94143     
2001 W 68TH ST HIALEAH
PALMETTO GENERAL HOPSITAL is not PALMETTO GENERAL HOPSITAL
33176 is not 33016     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2750 SW 37TH AVE MIAMI
SUITE 100 is not STE 100
33134 is not 33133

2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2200 ADA AVE CONWAY
SUITE 301 is not STE 301
72032 is not 72034     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
U

32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZA is not UF HEALTH CONGENITAL HEART CENTER MEDICA
32610 is not 32608     
2000 SW ARCHER RD GAINESVILLE
UF HEALTH CONGENITAL HEART CENTER MEDICAL PLAZ

...

45458 is not 45459     
1 W RIDGEWOOD AVE PARAMUS
SUITE 211 HACKENSACK UNIVERSITY MEDICAL GROUP is not SUITE 211 HACKENSACK UNIVERSITY MEDICAL
07630 is not 07652     
225 WILLIAMSON ST ELIZABETH
TRINITAS HOSPITAL is not TRINITAS HOSPITAL
07207 is not 07202     
210 E 64TH ST NEW YORK
4TH FLOOR is not 4TH FL
11065 is not 10065     
525 E 68TH ST NEW YORK
PULMONARY DISEASE is not PULMONARY DISEASE
10021 is not 10065     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
420 5TH AVE W HENDERSONVILLE
SUITE 300 is not STE 300
28793 is not 28739     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
1989 MIAMISBURG CENTERVILLE RD DAYTON
SUITE 204 is not STE 204
45342 is not 45459     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
1120 SE CARY PKWY CARY
SUITE 204 is not STE 204
27511 is not

28210 is not 28287     
525 E 68TH ST NEW YORK
EMERGENCY PHYSICIANS DEPT is not EMERGENCY PHYSICIANS DEPT
10021 is not 10065     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
587 BIELENBERG DR WOODBURY
SUITE 200 is not STE 200
55110 is not 55125     
525 E 68TH ST NEW YORK
EMERGENCY PHYSICIANS DEPT is not EMERGENCY PHYSICIANS DEPT
10021 is not 10065     
525 E 68TH ST NEW YORK
EMERGENCY PHYSICIANS DEPT is not EMERGENCY PHYSICIANS DEPT
10021 is not 10065     
525 E 68TH ST NEW YORK
PULMONARY DISEASE is not PULMONARY DISEASE
10021 is not 10065     
6060 PIEDMONT ROW DR S CHARLOTTE
FL 6 7 8 10 is not FL 6 7 8 10
28210 is not 28287     
1 W RIDGEWOOD AVE PARAMUS
SUITE 301 is not STE 301
07662 is not 07652     
587 BIELENBERG DR WOODBURY
SUITE 200 is not STE 200
55110 is not 55125     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
525 E 68TH ST NEW YORK
GYNECOLOGIC ONCOLOGY is not GYNECOLOGIC ONCOLOGY
10021 is n

SUITE 200 is not STE 200
37849 is not 37909     
130 FISHER RD BARRE
CENTRAL VERMONT is not 
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
4120 W MEMORIAL RD OKLAHOMA CITY
SUITE 300 is not STE 300
76312 is not 73120     
110 W 1325 N CEDAR CITY
SUITE 300 is not STE 300
84720 is not 84721     
110 W 1325 N CEDAR CITY
SUITE 300 is not STE 300
84720 is not 84721     
400 W LYNDON B JOHNSON FWY IRVING
SUITE 350 is not STE 350
75069 is not 75063     
985 BERKSHIRE BLVD WYOMISSING
SUITE 101 is not STE 101
19601 is not 19610     
1305 W 18TH ST SIOUX FALLS
PATHOLOGY CLINIC is not PATHOLOGY CLINIC
57104 is not 57105     
1305 W 18TH ST SIOUX FALLS
PATHOLOGY CLINIC is not 
57104 is not 57105     
1305 W 18TH ST SIOUX FALLS
PATHOLOGY CLINIC is not CHILD SPEC CLIN
57104 is not 57105     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is 

3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
3712 W 7TH ST FORT WORTH
MONTICELLO DIAGNOST is not MONTICELLO DIAGNOST
76104 is not 76107     
110 W 1325 N CEDAR CITY
SUITE 200 is not STE 200
84720 is not 84721     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not CENTRAL VERMONT
05602 is not 05641     
130 FISHER RD BARRE
CENTRAL VERMONT is not

2810 ASHLEY PHOSPHATE RD CHARLESTON
SUITE B 11 is not STE B11
29425 is not 29418     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
16463 BOONES FERRY RD LAKE OSWEGO
SUITE 300 is not STE 300
97305 is not 97035     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
1225 E WEISGARBER RD KNOXVILLE
SUITE 200 is not STE 200
37849 is not 37909     
3801 SCOTT AND WHITE DR KILLEEN
BAYLOR SCOTT AND WHITE CLINIC KILLE is not BAYLOR SCOTT AND WHITE CLINIC KILLE
76541 is not 76543     
1330 TAYLOR AT MARION ST COLUMBIA
SUITE 100 is not 
29201 is not 29220     
5126 W DAYBREAK PKWY SOUTH JORDAN
UUHC SOUTH JORDAN HEALTH CENTER is not UUHC SOUTH JORDAN HEALTH CENTER
84095 is not 84009     
2810 ASHLEY

In [ ]:
['entity_id', 'me#','comm_id', 'src_cat_code', 'addr_line2', 'addr_line1',
       'city_cd', 'state_cd', 'zip_x', 'ME', 'ENTITY_ID', 'addr_line_1',
       'addr_line_2_x', 'addr_city', 'addr_state',  'usage', 'load_type',
       'addr_type', 'addr_line_2_y', 'addr_line_3', 'addr_zip', 'addr_plus4',
       'addr_country', 'source', 'source_dtm', 'adrs_id', 'phone', 'zip_y']

In [ ]:
[['entity_id', 'me#', 'comm_id', 'usage', 'load_type_ind', 'addr_type',
       'addr_line_1', 'addr_line_3', 'addr_line_2', 'addr_city', 'addr_state',
       'addr_zip', 'addr_plus4', 'addr_country', 'source', 'source_dtm', 'phone_number']]

In [82]:
len(cred_addresses)

312618

In [84]:
cred_addresses = pd.merge(cred_addresses, ENTITY_TO_ME, on='entity_id')
cred_addresses.to_csv('../../Data/Credentialling/Data_Load.csv', index=False)